In [37]:
%load_ext autoreload
%autoreload 2
import sys
import collections
import numpy as np
import re
import os

sys.path.append("/home/rame/trl/examples/llama/")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [38]:
import os
import numpy as np
import torch
from datasets import load_dataset

from trl import PPOConfig, PPOTrainer, set_seed
from trl.core import LengthSampler

import llama_utils, ppo_utils, args_utils

MIN_SIZE = 100
MAX_SIZE = 1200

def build_news_dataset(
    tokenizer,
    split="train"
):
    """
    Args:
        dataset_name (`str`): "argilla/news-summary"
    """
    ds = load_dataset("argilla/news-summary", name="comparisons", split=split)
    raise NotImplementedError()
    return ds

In [56]:
dsv = load_dataset("argilla/news-summary", name="comparisons", split="test")

Found cached dataset parquet (/home/rame/.cache/huggingface/datasets/argilla___parquet/argilla--news-summary-46ccad7a40bceec1/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


In [40]:
script_args = args_utils.DefaultArgsPPO()
tokenizer = llama_utils.Tokenizer.load_tokenizer(script_args.base_model_name)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LLaMATokenizer'. 
The class this function is called from is 'LlamaTokenizer'.


In [57]:
dsv

Dataset({
    features: ['text', 'prediction', 'prediction_agent', 'annotation', 'annotation_agent', 'id', 'metadata', 'status', 'event_timestamp', 'metrics'],
    num_rows: 20417
})

In [68]:
def remove_duplicate(duplicated_dataset):
    initial_list = duplicated_dataset.map(lambda x: {"id": x['id']})
    _ , unique_indices = np.unique(initial_list["id"], return_index=True, axis=0)
    filtered_dataset = duplicated_dataset.select(unique_indices.tolist())
    return filtered_dataset

ds_deduplicated = remove_duplicate(ds_filtered)

In [86]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
reward_name = "OpenAssistant/reward-model-deberta-v3-base"
rank_model, tokenizer = AutoModelForSequenceClassification.from_pretrained(reward_name), AutoTokenizer.from_pretrained(reward_name)
question, answer = "Explain nuclear fusion like I am five", "Nuclear fusion is the process by which two or more protons and neutrons combine to form a single nucleus. It is a very important process in the universe, as it is the source of energy for stars and galaxies. Nuclear fusion is also a key process in the production of energy for nuclear power plants."
inputs = tokenizer(question, answer, return_tensors='pt')
score = rank_model(**inputs).logits[0].cpu().detach()
print(score)

tensor([0.5816])


In [88]:
inputs

{'input_ids': tensor([[    1, 19719,  3374, 11156,   334,   273,   481,   773,     2, 13495,
         11156,   269,   262,   568,   293,   319,   375,   289,   310, 55532,
           263, 64739,  4650,   264,   726,   266,   777, 21902,   260,   325,
           269,   266,   379,   539,   568,   267,   262,  4660,   261,   283,
           278,   269,   262,  1271,   265,   843,   270,  2906,   263, 24063,
           260, 13495, 11156,   269,   327,   266,   896,   568,   267,   262,
          1117,   265,   843,   270,  3374,   617,  2239,   260,     2]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
  

In [94]:
out_text = tokenizer.cls_token + question + tokenizer.sep_token + answer

# list(zip(inputs["input_ids"][0].tolist(), tokenizer.encode(out_text)[1:]))

In [102]:
print(len(inputs["input_ids"][0].tolist()))
print(len(tokenizer.encode(out_text)[1:]))

69
69


In [92]:
tokenizer.decode(**inputs)

TypeError: encode() missing 1 required positional argument: 'text'

In [84]:

    input_size_sampler = LengthSampler(
        2,
        8
    )

    def tokenize(sample):
        info_post = "-".join(sample["text"].replace("\n", " ").split("(Reuters) -")[1:]).strip()
        prompt_summary = llama_utils.Instructions.get_prompt_summary(post=info_post)
        size_prompt_summary = len(tokenizer.encode(prompt_summary)) - 1
        input_size = size_prompt_summary + input_size_sampler()
        choice = 0 # select the best summary
        response = sample["prediction"][choice]["text"].replace("\n", " ").replace(".", ",")
        sample["input_ids"] = tokenizer.encode(prompt_summary + response)[:input_size]
        sample["query"] = tokenizer.decode(sample["input_ids"])
        return sample

    ds_mapped = ds_deduplicated.map(tokenize, batched=False, load_from_cache_file=False)
    ds_mapped.set_format(type="torch")

In [85]:
ds_mapped["query"]

[' ### Instruction: Generate a one-sentence summary of this post. ### Input: A Macedonian court sentenced 33 ethnic Albanians to jail terms on Thursday, including seven given life sentences, after they were found guilty of plotting attacks and clashing with the police in a 2015 shootout in which 22 people were killed. The 33 were accused of being part of a group of Albanian gunmen involved in the May 9, 2015 shootout with police in the northern Macedonian town of Kumanovo, in which eight police officers were among those killed and dozens were wounded. Some of the defendants were from neighboring Kosovo, while others were from Macedonia s own Albanian minority.  The evidence offered by the prosecutor shows that the accused committed the criminal acts of terrorism and terrorist organization,  the presiding judge said.  The court was under heavy police protection, with helicopters flying overhead. The defendants protested while the verdict was being read, prompting the judge to order 33 o

In [78]:
ds_deduplicated["prediction"][0][0]["text"]

'Macedonia jails 33 ethnic Albanians for 2015 shootout'

In [66]:
ds_filtered = ds.filter(
    lambda x: x["text"] is not None and MIN_SIZE < len(x["text"]) < 2000 and
    x["id"] is not None,
    batched=False
)

In [67]:
ds_filtered

Dataset({
    features: ['text', 'prediction', 'prediction_agent', 'annotation', 'annotation_agent', 'id', 'metadata', 'status', 'event_timestamp', 'metrics'],
    num_rows: 459
})

In [48]:
ds

Dataset({
    features: ['text', 'prediction', 'prediction_agent', 'annotation', 'annotation_agent', 'id', 'metadata', 'status', 'event_timestamp', 'metrics'],
    num_rows: 304
})

In [42]:
ds["text"][0]

'PHNOM PENH (Reuters) - Sweden said on Tuesday it was stopping new aid for Cambodia, except in education and research, and would no longer support a reform programme after the main opposition party was outlawed by the Supreme Court at the government s request. The announcement marked the first concrete action by a European Union country in protest at a political crackdown in which veteran Prime Minister Hun Sen s main rival has also been arrested and civil rights groups and independent media attacked. The United States cut election funding and said it would take more punitive steps after last week s ban on the Cambodia National Rescue Party (CNRP). The European Union has also threatened action. Sweden s embassy in Phnom Penh said the country was reviewing its engagement with Cambodia.   We will not initiate any new government-to-government development cooperation agreements, except in the areas of education and research,  it said in a statement. As a consequence, it would be unable to 

In [3]:
# python3 llama_ppo_summary.py --sentiment_model Tristan/gpt2_reward_summarization --score_goal 0 --output_max_length 32 --ref_model none --base_model_name alexrame/alpaca-lora-7b-merged --peft_name lora
# python3 llama_ppo_summary.py --base_model_name alexrame/alpaca-lora-7b-merged --peft_name lora --ref_model none  --sentiment_model CogComp/bart-faithful-summary-detector --score_goal "1-0"

import os
import numpy as np
import torch
from datasets import load_dataset

from trl import PPOConfig, PPOTrainer, set_seed
from trl.core import LengthSampler

import llama_utils, ppo_utils, args_utils

MIN_SIZE = 100
MAX_SIZE = 1200

def build_dataset(
    tokenizer,
    dataset_name="openai/summarize_from_feedback",
    split="train"
):
    """
    Build dataset for training. This builds the dataset from `load_dataset`, one should
    customize this function to train the model on its own dataset.

    Args:
        dataset_name (`str`):
            The name of the dataset to be loaded.
            Either "openai/summarize_from_feedback" or "argilla/news-summary"
    Returns:
        dataloader (`torch.utils.data.DataLoader`):
            The dataloader for the dataset.
    """
    ds = load_dataset(dataset_name, name="comparisons", split=split)
    ds = ds.filter(
        lambda x: x["info"]["post"] is not None and MIN_SIZE < len(x["info"]["post"]) <
        MAX_SIZE and x['info']["id"] is not None,
        batched=False
    )

    def remove_duplicate(duplicated_dataset):
        initial_list = duplicated_dataset.map(lambda x: {"id": x['info']["id"]})
        _ , unique_indices = np.unique(initial_list["id"], return_index=True, axis=0)
        filtered_dataset = duplicated_dataset.select(unique_indices.tolist())
        return filtered_dataset

    ds = remove_duplicate(ds)

    input_size_sampler = LengthSampler(
        2,
        8
    )

    def tokenize(sample):
        info_post = sample["info"]["post"].replace("\n", " ")
        prompt_summary = llama_utils.Instructions.get_prompt_summary(post=info_post)
        size_prompt_summary = len(tokenizer.encode(prompt_summary)) - 1
        input_size = size_prompt_summary + input_size_sampler()
        response = sample["info"]["response"].replace("\n", " ")
        sample["input_ids"] = tokenizer.encode(prompt_summary + response)[:input_size]
        sample["query"] = tokenizer.decode(sample["input_ids"])
        return sample

    ds = ds.map(tokenize, batched=False, load_from_cache_file=False)
    ds.set_format(type="torch")
    return ds

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LLaMATokenizer'. 
The class this function is called from is 'LlamaTokenizer'.


In [6]:
dataset_name="openai/summarize_from_feedback"
split="train"

In [7]:
ds = load_dataset(dataset_name, name="comparisons", split=split)
ds = ds.filter(
    lambda x: x["info"]["post"] is not None and MIN_SIZE < len(x["info"]["post"]) <
    MAX_SIZE and x['info']["id"] is not None,
    batched=False
)

Found cached dataset summarize_from_feedback (/home/rame/.cache/huggingface/datasets/openai___summarize_from_feedback/comparisons/0.0.0/483f970ceb55b926b0a087ef4f678ab1b089bc8174a107a452c6152e88af7ff0)


In [19]:
def remove_duplicate(duplicated_dataset):
    initial_list = duplicated_dataset.map(lambda x: {"id": x['info']["id"]})
    _ , unique_indices = np.unique(initial_list["id"], return_index=True, axis=0)
    filtered_dataset = duplicated_dataset.select(unique_indices.tolist())
    return filtered_dataset

ds = remove_duplicate(ds)

In [21]:

input_size_sampler = LengthSampler(
    2,
    8
)

In [22]:
def tokenize(sample):
    info_post = sample["info"]["post"].replace("\n", " ")
    prompt_summary = llama_utils.Instructions.get_prompt_summary(post=info_post)
    size_prompt_summary = len(tokenizer.encode(prompt_summary)) - 1
    input_size = size_prompt_summary + input_size_sampler()
    choice = sample["choice"]
    response = sample["summaries"][choice]["text"].replace("\n", " ")
    sample["input_ids"] = tokenizer.encode(prompt_summary + response)[:input_size]
    sample["query"] = tokenizer.decode(sample["input_ids"])
    return sample

new_ds = ds.map(tokenize, batched=False, load_from_cache_file=False)

In [24]:
new_ds["query"]

[" ### Instruction: Generate a one-sentence summary of this post. ### Input: My boyfriend and I have been together for five years and would like to publicize our engagement in 6 months. His family feels dating is a sin hence I have never met them. We rather not shock his parents so we felt me being introduced as a friend six months before hand is a good idea.   She will meet me and another friend of his as she drops some food off nexr week at my boyfriend's apartment. I don't think the exchange will last for more than five minutes but as expected I am very nervous. I do not want it to become evident during the exchange that I am his girlfriend nor do I want her to see me as someone not swell enough to be his son's friend.  How should I do this? I want to make it perfect. Any tips?   Pardon the throwaway; my boyfriend loves Reddit and I rather him not know of my nerves. ### Response:  How do",
 " ### Instruction: Generate a one-sentence summary of this post. ### Input: Hi Runnit.  On Th

In [32]:
        ds.set_format("pandas")

In [33]:
        df_batch = ds[:10]

{'id': 't3_100kfa', 'post': "My boyfriend and I have been together for five years and would like to publicize our engagement in 6 months. His family feels dating is a sin hence I have never met them. We rather not shock his parents so we felt me being introduced as a friend six months before hand is a good idea. \n\nShe will meet me and another friend of his as she drops some food off nexr week at my boyfriend's apartment. I don't think the exchange will last for more than five minutes but as expected I am very nervous. I do not want it to become evident during the exchange that I am his girlfriend nor do I want her to see me as someone not swell enough to be his son's friend.\n\nHow should I do this? I want to make it perfect. Any tips? \n\nPardon the throwaway; my boyfriend loves Reddit and I rather him not know of my nerves.", 'title': "I [mid-twenties female] will be meeting my boyfriend [mid-twenties male]'s Islamic mom next week. I will only be introduced as a friend. How can I n

[None, None, None, None, None, None, None, None, None, None]

In [30]:
        print(df_batch["query"][:3])

KeyError: 'query'

In [27]:
ds["summaries"][1][0]["text"].replace(".", ",")

" I'm sick,  Should I start running again by Tuesday?  Is it possible to feel better with long runs?"

In [18]:
ds["choice"]

[0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,


In [17]:
ds["summaries"][0][0]

{'text': " RA owes me money, made me buy her toys in a fundraiser, bought my sister's toys, now owes me 6 dollars.  How can I get her to pay me?",
 'policy': 'sup1',
 'note': None}

In [13]:
np.array(initial_list["id"])

array(['t3_52mb8y', 't3_52mb8y', 't3_52mb8y', ..., 't3_rxm7p', None, None],
      dtype=object)

In [14]:
_ , unique_indices = np.unique(np.array(initial_list["id"]), return_index=True, axis=0)

TypeError: The axis argument to unique is not supported for dtype object

In [7]:
ds

Dataset({
    features: ['info', 'summaries', 'choice', 'worker', 'batch', 'split', 'extra'],
    num_rows: 36861
})

In [4]:
script_args.num_samples=10

In [5]:
print(llama_utils.Naming.str_dict(script_args.__dict__))



########################################
base_model_name    decapoda-research/llama-7b-hf
every              0
num_samples        10
output_max_length  32
peft_names         ['tloen/alpaca-lora-7b']
prompt             medium
sentiment_models   ['Tristan/gpt2_reward_summarization', 'CogComp/bart-faithful-summary-detector']
verbose            0
########################################



In [6]:
# 1. load dataset and tokenizers
tokenizer = llama_utils.Tokenizer.load_tokenizer(script_args.base_model_name)
if script_args.num_samples == -1:
    query_tensors = Samples.get_fake_samples_summary()
else:
    query_tensors = Samples.get_samples_summary(bs=script_args.num_samples)
print("Example input:", tokenizer.decode(query_tensors[0]).replace("\n", " "))

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LLaMATokenizer'. 
The class this function is called from is 'LlamaTokenizer'.
Found cached dataset summarize_from_feedback (/home/rame/.cache/huggingface/datasets/openai___summarize_from_feedback/comparisons/0.0.0/483f970ceb55b926b0a087ef4f678ab1b089bc8174a107a452c6152e88af7ff0)
Loading cached processed dataset at /home/rame/.cache/huggingface/datasets/openai___summarize_from_feedback/comparisons/0.0.0/483f970ceb55b926b0a087ef4f678ab1b089bc8174a107a452c6152e88af7ff0/cache-6ff377a28a698e8f.arrow
Loading cached processed dataset at /home/rame/.cache/huggingface/datasets/openai___summarize_from_feedback/comparisons/0.0.0/483f970ceb55b926b0a087ef4f678ab1b089bc8174a107a452c6152e88af7ff0/cache-d324764c5131aff5.arrow
Loading cached processed dataset at /home/rame/.cache/huggingface/dat

Example input:  Below is an instruction that describes a task. Write a response that appropriately completes the request. ### Instruction: Generate a one-sentence summary of this post.  ### Input: Me [24] Him [28], relationship of 6 months.  It's not that my boyfriend doesn't respect me.  It's that I want him to respect me even more.  I want him to admire me and for him to see me in my best light as often as possible.  Maybe this is non-sequitor, but there are so many TV wives who are not very respect-worthy.  Wives are often seen as kind of a drag.  I don't want to be that ever in his mind.  What can I do to be a woman worthy of great respect in his eyes?  Thanks for your input!  ### Response:


In [8]:
for i in range(10):
    print("Example input:", tokenizer.decode(query_tensors[i]).replace("\n", " "))

Example input:  Below is an instruction that describes a task. Write a response that appropriately completes the request. ### Instruction: Generate a one-sentence summary of this post.  ### Input: Me [24] Him [28], relationship of 6 months.  It's not that my boyfriend doesn't respect me.  It's that I want him to respect me even more.  I want him to admire me and for him to see me in my best light as often as possible.  Maybe this is non-sequitor, but there are so many TV wives who are not very respect-worthy.  Wives are often seen as kind of a drag.  I don't want to be that ever in his mind.  What can I do to be a woman worthy of great respect in his eyes?  Thanks for your input!  ### Response:
Example input:  Below is an instruction that describes a task. Write a response that appropriately completes the request. ### Instruction: Generate a one-sentence summary of this post.  ### Input: I used to walk to school in the 3rd grade.  At the halfway mark, I felt a pressure bubbling in my g

In [9]:
script_args.sentiment_models

['Tristan/gpt2_reward_summarization', 'CogComp/bart-faithful-summary-detector']

In [10]:
sentiment_model = 'CogComp/bart-faithful-summary-detector'

In [16]:
from transformers import pipeline

In [13]:
tokenizer_name=llama_utils.Tokenizer.load_tokenizer_name(sentiment_model)

In [14]:
tokenizer_name

'CogComp/bart-faithful-summary-detector'

In [18]:
pipeline??

In [20]:
pipe = pipeline(
    "text-classification",
    model=sentiment_model,
    device=device,
    tokenizer=tokenizer_name,
    num_labels=2

)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'HALLUCINATED', '1': 'FAITHFUL'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'HALLUCINATED', '1': 'FAITHFUL'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'HALLUCINATED', '1': 'FAITHFUL'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'HALLUCINATED', '1': 'FAITHFUL'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'HALLUCINATED', '1': 'FAITHFUL'}. The number of labels wil be overwritten to 2.


In [ ]:
ds = load_dataset(dataset_name, name="comparisons", split=split)
# dsn = load_dataset()
ds = ds.filter(lambda x: x["info"]["post"] is not None and len(x["info"]["post"]) < MAX_SIZE, batched=False)

In [ ]:
ds["info"]

In [ ]:
initial_list = ds.map(lambda x: 

In [ ]:
unique_indices

In [ ]:
filtered_dataset["info"]

In [ ]:
unique_indices = np.unique(initial_list, return_index=True, axis=0)

In [ ]:
initial_list

In [ ]:

_ , unique_indices = np.unique(initial_list, return_index=True, axis=0)
filtered_dataset = dataset.select(unique_indices.tolist())

In [ ]:
type(ds)

In [ ]:
MAX_SIZE = 500 # 1200

def build_dataset(
    tokenizer,
    dataset_name="openai/summarize_from_feedback",
    split="train"
):
    """
    Build dataset for training. This builds the dataset from `load_dataset`, one should
    customize this function to train the model on its own dataset.

    Args:
        dataset_name (`str`):
            The name of the dataset to be loaded.
            Either "openai/summarize_from_feedback" or "argilla/news-summary"
    Returns:
        dataloader (`torch.utils.data.DataLoader`):
            The dataloader for the dataset.
    """
    # load summary datasets
    ds = load_dataset(dataset_name, name="comparisons", split=split)
    # dsn = load_dataset()
    ds = ds.filter(lambda x: x["info"]["post"] is not None and len(x["info"]["post"]) < MAX_SIZE, batched=False)

    def tokenize(sample):
        info_post = sample["info"]["post"].replace("\n", " ")
        text = llama_utils.Instructions.get_prompt_summary(post=info_post, response="")
        sample["input_ids"] = tokenizer.encode(text)[:-2]
        sample["query"] = tokenizer.decode(sample["input_ids"])
        return sample

    ds = ds.map(tokenize, batched=False)
    ds.set_format(type="torch")
    return ds

In [ ]:
llama_utils.Instructions.get_prompt_summary(post="test post", response="response")

In [ ]:
tokenizer = llama_utils.Tokenizer.load_tokenizer(script_args.base_model_name)

In [ ]:
# 2. load dataset
if script_args.num_samples == -1:
    query_tensors = Samples.get_fake_samples_summary()
else:
    query_tensors = Samples.get_samples_summary(bs=script_args.num_samples)
print("Example:", tokenizer.decode(query_tensors[0]))

In [ ]:
ds = llama_ppo_summary.build_dataset(
    tokenizer=tokenizer, split="validation"
)

In [ ]:
type(ds)

In [ ]:
ds.set_format("pandas")

In [ ]:
from datasets import load_dataset


In [ ]:
ds = load_dataset("openai/summarize_from_feedback", name="comparisons", split="validation")


In [ ]:
# load imdb with datasets
# dsn = load_dataset()
ds = ds.filter(lambda x: len(x["info"]["post"]) < 500, batched=False)

In [ ]:
def tokenize(sample):
    sample["info_post"] = sample["info"]["post"].replace("\n", " ")
    text = llama_utils.Instructions.get_prompt_summary(post=sample["info_post"], response="")
    sample["input_ids"] = tokenizer.encode(text)[:-2]
    sample["query"] = tokenizer.decode(sample["input_ids"])
    return sample

ds = ds.map(tokenize, batched=False)

In [ ]:
ds["query"]

In [ ]:
ds.set_format(type="torch")

In [ ]:
ds["query"]

In [ ]:
df_batch = ds[:2]
query_tensors = df_batch['input_ids'].tolist()

In [ ]:
print("Example:", tokenizer.decode(query_tensors[0]))

In [ ]:
        ds.set_format("pandas")

In [ ]:
ds = llama_ppo_summary.build_dataset(
    tokenizer=tokenizer, split="validation"
)

In [ ]:
ds.set_format("pandas")

In [ ]:
ds[0]

In [ ]:
query_tensors = ds['input_ids'].tolist()

In [ ]:
query_tensors[0]

In [ ]:
print("Example:", tokenizer.decode(query_tensors[0]))

In [ ]:
ds[0]["query"]

In [ ]:
if script_args.num_samples == -1:
    query_tensors = Samples.get_fake_samples_summary()
else:
    query_tensors = Samples.get_samples_summary(bs=script_args.num_samples)

In [ ]:
tokenizer.decode(query_tensors[0])

In [ ]:
true_query_tensors = Samples.get_samples_summary(bs=2)

In [ ]:
true_query_tensors[0]

In [ ]:
tokenizer.decode(true_query_tensors[0])